#Imports

In [1]:
!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TrainingArguments, Trainer
from transformers import pipeline
import torch

In [6]:
!apt-get install -y build-essential cmake
!git clone https://github.com/ggerganov/llama.cpp
%cd llama.cpp
!make

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Cloning into 'llama.cpp'...
remote: Enumerating objects: 49322, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 49322 (delta 14), reused 6 (delta 6), pack-reused 49291 (from 3)
Receiving objects: 100% (49322/49322), 102.93 MiB | 22.27 MiB/s, done.
Resolving deltas: 100% (35570/35570), done.
/content/llama.cpp
Makefile:2: *** The Makefile build is deprecated. Use the CMake build instead. For more details, see https://github.com/ggml-org/llama.cpp/blob/master/docs/build.md.  Stop.


#Tokenize Data

In [3]:
dataset = load_dataset("json", data_files="/content/drive/MyDrive/finetune_data.jsonl", split="train")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [4]:
tokenizer.pad_token = tokenizer.eos_token

In [5]:
def tokenize_function(examples):
    tokenized = tokenizer(examples["completion"], truncation=True, padding="max_length", max_length=512)
    tokenized["labels"] = tokenized["input_ids"]
    return tokenized

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/27687 [00:00<?, ? examples/s]

#Model and trainer

In [6]:
model = GPT2LMHeadModel.from_pretrained("gpt2")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [7]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    logging_dir="./logs",
    logging_steps=500,
    save_steps=1000,
    save_total_limit=2,
    report_to="wandb"
)

In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets
)

#Train Model

In [9]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: r77266071 (r77266071-vellore-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
500,0.791200
1000,0.723500
1500,0.754300
2000,0.693100
2500,0.672700
3000,0.644600
3500,0.653200
4000,0.655800
4500,0.671700
5000,0.658800


TrainOutput(global_step=20766, training_loss=0.6149305754930404, metrics={'train_runtime': 13353.0857, 'train_samples_per_second': 6.22, 'train_steps_per_second': 1.555, 'total_flos': 2.1703177469952e+16, 'train_loss': 0.6149305754930404, 'epoch': 3.0})

#Save Model

In [10]:
trainer.save_model("finetuned_gpt2")
tokenizer.save_pretrained("finetuned_gpt2")

('finetuned_gpt2/tokenizer_config.json',
 'finetuned_gpt2/special_tokens_map.json',
 'finetuned_gpt2/vocab.json',
 'finetuned_gpt2/merges.txt',
 'finetuned_gpt2/added_tokens.json')

In [11]:
!cp -r /content/finetuned_gpt2 /content/drive/MyDrive/finetuned_gpt2

#Test

In [17]:
model = GPT2LMHeadModel.from_pretrained("finetuned_gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("finetuned_gpt2")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

prompt = "Chapter 1: My family and I began our journey into the unknown. Explain this chapter as a summary"
generated_text = generator(prompt, max_length=900, num_return_sequences=1)
print(generated_text[0]['generated_text'])

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Chapter 1: My family and I began our journey into the unknown. Explain this chapter as a summary, based on the text you have been given. As an adult, the time you spend in the school is divided between doing a project (working on a project) and studying the facts. This chapter introduces learners to the concept of time, and introduces them to various aspects of travel, including the concept of time, time management, and money – concepts that are useful, but not essential. Students will be curious to gather information about travel from these pages, and then explore various aspects of that journey. After this, they will experience travel as a natural journey — in a sense of adventure, adventure, adventure — and to travel as a source of discovery and guidance, but also as a guide and inspiration to others. The adventure-orientation section, along with the travel-related topics will become the key focus of this chapter. The chapter covers many topics such as science, nature, life sciences

#Lightweight Model

In [13]:
!python3 convert_hf_to_gguf.py /content/drive/MyDrive/Edubuddy/finetuned_gpt2 --outfile /content/finetuned_gpt2.gguf

INFO:hf-to-gguf:Loading model: finetuned_gpt2
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model part 'model.safetensors'
INFO:hf-to-gguf:blk.0.attn_qkv.bias,       torch.float32 --> F32, shape = {2304}
INFO:hf-to-gguf:blk.0.attn_qkv.weight,     torch.float32 --> F16, shape = {768, 2304}
INFO:hf-to-gguf:blk.0.attn_output.bias,    torch.float32 --> F32, shape = {768}
INFO:hf-to-gguf:blk.0.attn_output.weight,  torch.float32 --> F16, shape = {768, 768}
INFO:hf-to-gguf:blk.0.attn_norm.bias,      torch.float32 --> F32, shape = {768}
INFO:hf-to-gguf:blk.0.attn_norm.weight,    torch.float32 --> F32, shape = {768}
INFO:hf-to-gguf:blk.0.ffn_norm.bias,       torch.float32 --> F32, shape = {768}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,     torch.float32 --> F32, shape = {768}
INFO:hf-to-gguf:blk.0.ffn_up.bias,         torch.float32 --> F32, shape = {3072}
INFO:hf-to-gguf:blk.0.ffn_up.weight,       torch.float32

In [15]:
!cp -r /content/finetuned_gpt2.gguf /content/drive/MyDrive/Edubuddy